In [1]:
import openai
import polars as pl
import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [2]:
# Загрузка данных в Polars
df = pl.read_csv('rag.csv')
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-small"
# )
# print(response.data[0].embedding)


In [3]:
df

questions,example
str,str
"""Двигатель (двз, мотор, движок)""","""1: Тип топлива: Бензиновый ил…"
"""АКПП (автоматическая коробка п…","""1: Комплектация: Коробка с бл…"
"""МКПП""","""1: Уточнить год выпуска машины…"
"""Кулак""","""1: Диаметр заднего сальника: …"
"""Привод передний""","""1: Уточнить год выпуска машины…"
…,…
"""трубка гур""","""1: Год выпуска: Каков год выпу…"
"""фонарь задний дорестайл (красн…","""Если до 99 то - скорее всего у…"
"""Фонарь задний правый""","""1: Фонари у вас красно белые и…"


In [4]:
# Генерация эмбеддингов для вопросов
def generate_embeddings(texts):
    response = openai.embeddings.create(
        input=texts,
        model="text-embedding-3-small"
    )
    return [item.embedding for item in response.data]

# Получение эмбеддингов
questions = df['questions'].to_list()
print(questions)




['Двигатель (двз, мотор, движок)', 'АКПП (автоматическая коробка передач)', 'МКПП', 'Кулак', 'Привод передний', 'Суппорт', 'Редуктор задний (задний мост)', 'Редуктор передний (передний мост)', 'Редуктор рулевой (Рулевая колонка)', 'Крылья передние', 'Зеркало', 'Фары', 'Задние фонари', 'Головка блока цилиндров (ГБЦ)', 'Форсунки', 'Выпускной коллектор', 'Дроссель', 'Клапанная крышка', 'ДМРВ (датчик массового расхода воздуха)', 'ГТЦ (главный тормозной цилиндр)', 'Радиатор основной', 'Радиатор кондиционера', 'Компрессор кондиционера', 'Осушитель кондиционера', 'Трубки кондиционера', 'Решетка радиатора', 'Бампер', 'Усилитель бампера', 'Трапеция дворников', 'Моторчик дворников', 'Жабо', 'Капот', 'Блок цилиндров (низ мотора)', 'Коленвал', 'Поршни', 'Шатун', 'Шкив коленвала', 'Шестерня коленвала', 'Генератор', 'Стартер', 'Маховик', 'Лямбда', 'Круиз-контроль', 'Вискамуфта', 'Защита картера', 'Поддон картера', 'Скоба суппорта', 'ЭБУ (мозги, блок управления двигателем)', 'Коса проводка', 'ДХХ (да

In [5]:
embeddings = generate_embeddings(questions)
print(embeddings)


[[-0.0047150361351668835, 0.06534039974212646, -0.09882638603448868, -0.011547226458787918, 0.02738702856004238, 0.020685946568846703, -0.03336944431066513, 0.015966055914759636, 0.016587603837251663, 0.004460103809833527, 0.02286137081682682, 0.007609126623719931, 0.014421893283724785, 0.0025031939148902893, 0.015422198921442032, 0.04075034335255623, -0.022725407034158707, -0.021831929683685303, 0.0017019775696098804, -0.005962991621345282, 0.041410740464925766, -0.035039857029914856, 0.02585257776081562, -0.013946019113063812, 0.041799210011959076, 0.005283171311020851, 0.021462883800268173, -0.021540578454732895, -0.0070798383094370365, -0.06973009556531906, 0.038827426731586456, -0.019967280328273773, 0.029659569263458252, -0.040245335549116135, 0.007244937587529421, 0.011780306696891785, -0.03698220103979111, -0.0052249012514948845, 0.008381207473576069, 0.025638921186327934, 0.004071635194122791, -0.05811488628387451, -0.02659066766500473, 0.036166418343782425, 0.0048340046778321

In [6]:
# Сохранение эмбеддингов в Polars
df = df.with_columns(pl.Series("embeddings", embeddings))
print(df)


shape: (83, 3)
┌───────────────────────────────────┬───────────────────────────────────┬──────────────────────────┐
│ questions                         ┆ example                           ┆ embeddings               │
│ ---                               ┆ ---                               ┆ ---                      │
│ str                               ┆ str                               ┆ list[f64]                │
╞═══════════════════════════════════╪═══════════════════════════════════╪══════════════════════════╡
                  ┆ [-0.004715, 0.06534, …   │оплива:
│                                   ┆ Бензиновый ил…                    ┆ 0.02806…                 │
                 ┆ [-0.002357, 0.000476, …  │лектация:
│                                   ┆ Коробка с бл…                     ┆ 0.0333…                  │
│ МКПП                              ┆ 1: Уточнить год выпуска машины…   ┆ [-0.019924, 0.005367, …  │
│                                   ┆                               

In [7]:
df

questions,example,embeddings
str,str,list[f64]
"""Двигатель (двз, мотор, движок)""","""1: Тип топлива: Бензиновый ил…","[-0.004715, 0.06534, … 0.028067]"
"""АКПП (автоматическая коробка п…","""1: Комплектация: Коробка с бл…","[-0.002357, 0.000476, … 0.033364]"
"""МКПП""","""1: Уточнить год выпуска машины…","[-0.019924, 0.005367, … 0.005217]"
"""Кулак""","""1: Диаметр заднего сальника: …","[0.017982, 0.00771, … -0.020407]"
"""Привод передний""","""1: Уточнить год выпуска машины…","[0.001518, 0.017535, … -0.010582]"
…,…,…
"""трубка гур""","""1: Год выпуска: Каков год выпу…","[-0.028833, 0.027844, … -0.011938]"
"""фонарь задний дорестайл (красн…","""Если до 99 то - скорее всего у…","[0.002956, 0.026989, … -0.018646]"
"""Фонарь задний правый""","""1: Фонари у вас красно белые и…","[0.001266, 0.055596, … -0.005571]"


In [9]:
df.filter(df['questions'] == 'Двигатель (двз, мотор, движок)')


questions,example,embeddings
str,str,list[f64]
"""Двигатель (двз, мотор, движок)""","""1: Тип топлива: Бензиновый ил…","[-0.004715, 0.06534, … 0.028067]"


In [10]:
from typing import List
from scipy import spatial

def distances_from_embeddings(
    query_embedding: List[float],
    embeddings: List[List[float]],
    distance_metric="cosine",
) -> List[List]:
    distance_metrics = {
        "cosine": spatial.distance.cosine,
        "L1": spatial.distance.cityblock,
        "L2": spatial.distance.euclidean,
        "Linf": spatial.distance.chebyshev,
    }
    distances = [
        distance_metrics[distance_metric](query_embedding, embedding)
        for embedding in embeddings
    ]
    return distances
def find_similar(query, embeddings, questions, top_n=5, threshold=0.7):
    """
    Ищет похожие вопросы на основе эмбеддингов.

    :param query: Строка запроса.
    :param embeddings: Список эмбеддингов для всех вопросов.
    :param questions: Список всех вопросов.
    :param top_n: Максимальное количество возвращаемых результатов.
    :param threshold: Минимальный порог сходства для включения в результаты.
    :return: Список кортежей (вопрос, сходство).
    """
    emb = generate_embeddings([query])
    query_embedding = emb[0]
    similarities = distances_from_embeddings(query_embedding, embeddings, distance_metric="cosine")
    
    # Для косинусного расстояния: меньшие значения означают большую схожесть
    # Поэтому сортируем по возрастанию
    sorted_indices = np.argsort(similarities)
    
    results = []
    count = 0
    for idx in sorted_indices:
        similarity = 1 - similarities[idx]  # Преобразуем расстояние в сходство
        if similarity >= threshold:
            results.append((questions[idx], similarity))
            count += 1
            if count >= top_n:
                break
    return results


In [11]:
# Пример запроса
query = "детали автомобиля"
similar_questions = find_similar(query, embeddings, questions, threshold=0.2)
anwers = [i[0] for i in similar_questions]
print('\n'.join(anwers))


крыло переднее
задний дворник мотор рестайл
Блок цилиндров (низ мотора)
Привод передний
Крылья передние


In [12]:
anwers

['крыло переднее',
 'задний дворник мотор рестайл',
 'Блок цилиндров (низ мотора)',
 'Привод передний',
 'Крылья передние']

In [13]:
df.filter(
    df['questions'].is_in(anwers)
)['example']

example
str
"""1: Уточнить год выпуска машины…"
"""1: С расширителями или без? 2…"
"""1: Комплектация: Голый или в …"
"""1: Год выпуска: Каков год выпу…"
"""1: ""Вам надо под расширитель и…"


In [15]:
# Сохранение в файл (опционально)
df.write_parquet('embeddings.parquet')


In [18]:
df.write_excel('embeddings.xlsx')

In [17]:
pl.read_parquet('embeddings.parquet')


questions,example,embeddings
str,str,list[f64]
"""Двигатель (двз, мотор, движок)""","""1: Тип топлива: Бензиновый ил…","[-0.004715, 0.06534, … 0.028067]"
"""АКПП (автоматическая коробка п…","""1: Комплектация: Коробка с бл…","[-0.002357, 0.000476, … 0.033364]"
"""МКПП""","""1: Уточнить год выпуска машины…","[-0.019924, 0.005367, … 0.005217]"
"""Кулак""","""1: Диаметр заднего сальника: …","[0.017982, 0.00771, … -0.020407]"
"""Привод передний""","""1: Уточнить год выпуска машины…","[0.001518, 0.017535, … -0.010582]"
…,…,…
"""трубка гур""","""1: Год выпуска: Каков год выпу…","[-0.028833, 0.027844, … -0.011938]"
"""фонарь задний дорестайл (красн…","""Если до 99 то - скорее всего у…","[0.002956, 0.026989, … -0.018646]"
"""Фонарь задний правый""","""1: Фонари у вас красно белые и…","[0.001266, 0.055596, … -0.005571]"
